In [1]:
%set_env TOKENIZERS_PARALLELISM=false
!pip install esm
!pip install py3Dmol
!pip install sadie-antibody biotite
!pip install biopython

env: TOKENIZERS_PARALLELISM=false
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.8/35.8 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 45.8 MB/s eta 0:00:00
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.2/152.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.5/819.5 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.1/174.1 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 65.

Steps:
1. Download 3 pdb files with nanobodies from rcsb.org
2. Extract nanobody sequences, numerate them
3. Extract FR regions - both coordinates and aminoacids - fix them and use as a templates
4. For each of the templates generate N full sequences (with varying length of CDRs) and corresponding atomic structures
5. Download results from Google Colab


In [2]:
# imports
import pandas as pd
import numpy as np
from pathlib import Path
import os
import random
import torch
from datetime import datetime
def timestamp():
    return datetime.now().strftime("%Y-%m-%d_%H:%M:%S")

current_timestamp = timestamp()


def set_seed(seed: int = 42) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")

RANDOM_SEED = 3407  # 42  # 3407
set_seed(RANDOM_SEED)


Random seed set as 3407


In [3]:
# define constants
SAVEDIR = Path("esm3_generated_nanobodies")
SAVEDIR.mkdir(exist_ok=True)
TEMPLATEDIR = SAVEDIR / "templates"
TEMPLATEDIR.mkdir(exist_ok=True)

vhh_region_names = [
    'fwr1', 'cdr1',
    'fwr2', 'cdr2',
    'fwr3', 'cdr3',
    'fwr4'
]

In [4]:
# download_pdb_files
template_list = ["4krl", "4krn", "4krp"]
from Bio.PDB.PDBList import PDBList
PDBList().download_pdb_files(
    template_list,
    file_format="pdb",
    pdir=TEMPLATEDIR.as_posix()
)



In [5]:
# from Bio.PDB.PDBParser import PDBParser
from Bio import SeqIO

#parser = PDBParser(PERMISSIVE=1)
#structure = parser.get_structure(pdbid, filename.as_posix())


all_data = []
for pdbid in template_list:
    filename = TEMPLATEDIR / f"pdb{pdbid}.ent"
    id2dbxrefs = {record.id: record.dbxrefs for record in SeqIO.parse(filename.as_posix(), "pdb-seqres")}
    for record in SeqIO.parse(filename.as_posix(), "pdb-atom"):
        record_id = record.id
        chain_id = record.annotations["chain"]
        # print(record.dbxrefs)
        sequence = str(record.seq)
        all_data.append({
            'pdbid': pdbid,
            'record_id': record_id,
            'chain_id': chain_id,
            'dbxrefs': id2dbxrefs[record_id],#record.dbxrefs,
            'sequence': sequence,
            'filename': filename.as_posix()
        })
# correct way would be to extract sequence corresponding to residues

/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 2928.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 2940.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 2981.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 2995.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 15692.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain D is discontinuous at line 15776.
  warnings.warn(
/usr/local/lib/python3.10/dist-p

In [6]:
# record.description


In [7]:
templates_df = pd.DataFrame(all_data)
templates_df

,pdbid,record_id,chain_id,dbxrefs,sequence,filename
0,4krl,4KRL:A,A,"[UNP:P00533, UNP:EGFR_HUMAN]",LEEKKVCNGIGIGEFKDSLSINATNIKHFKNCTSISGDLHILPVAF...,esm3_generated_nanobodies/templates/pdb4krl.ent
1,4krl,4KRL:B,B,"[PDB:4KRL, PDB:4KRL]",QVKLEESGGGSVQTGGSLRLTCAASGRTSRSYGMGWFRQAPGKERE...,esm3_generated_nanobodies/templates/pdb4krl.ent
2,4krn,4KRN:A,A,"[PDB:4KRN, PDB:4KRN]",QVQLQESGGGLVQPGGSLRLSCAASGRTFSSYAMGWFRQAPGKQRE...,esm3_generated_nanobodies/templates/pdb4krn.ent
3,4krp,4KRP:A,A,"[UNP:P00533, UNP:EGFR_HUMAN]",KKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYVQRN...,esm3_generated_nanobodies/templates/pdb4krp.ent
4,4krp,4KRP:B,B,"[PDB:4KRP, PDB:4KRP]",EVQLVESGGGLXXXXXXXXLSCAASGRTFSSYAMGWFRQAPGKERE...,esm3_generated_nanobodies/templates/pdb4krp.ent
5,4krp,4KRP:C,C,"[PDB:4KRP, PDB:4KRP]",DILLTQSPVILSVSPGERVSFSCRASQSIGTNIHWYQQRTNGSPRL...,esm3_generated_nanobodies/templates/pdb4krp.ent
6,4krp,4KRP:D,D,"[PDB:4KRP, PDB:4KRP]",QVQLKQSGPGLVQPSQSLSITCTVSGFSLTNYGVHWVRQSPGKGLE...,esm3_generated_nanobodies/templates/pdb4krp.ent


In [8]:
templates_df['isEGFR'] = templates_df.dbxrefs.apply(lambda xs: "UNP:EGFR_HUMAN" in xs)

In [9]:
templates_df[~templates_df.isEGFR]

,pdbid,record_id,chain_id,dbxrefs,sequence,filename,isEGFR
1,4krl,4KRL:B,B,"[PDB:4KRL, PDB:4KRL]",QVKLEESGGGSVQTGGSLRLTCAASGRTSRSYGMGWFRQAPGKERE...,esm3_generated_nanobodies/templates/pdb4krl.ent,False
2,4krn,4KRN:A,A,"[PDB:4KRN, PDB:4KRN]",QVQLQESGGGLVQPGGSLRLSCAASGRTFSSYAMGWFRQAPGKQRE...,esm3_generated_nanobodies/templates/pdb4krn.ent,False
4,4krp,4KRP:B,B,"[PDB:4KRP, PDB:4KRP]",EVQLVESGGGLXXXXXXXXLSCAASGRTFSSYAMGWFRQAPGKERE...,esm3_generated_nanobodies/templates/pdb4krp.ent,False
5,4krp,4KRP:C,C,"[PDB:4KRP, PDB:4KRP]",DILLTQSPVILSVSPGERVSFSCRASQSIGTNIHWYQQRTNGSPRL...,esm3_generated_nanobodies/templates/pdb4krp.ent,False
6,4krp,4KRP:D,D,"[PDB:4KRP, PDB:4KRP]",QVQLKQSGPGLVQPSQSLSITCTVSGFSLTNYGVHWVRQSPGKGLE...,esm3_generated_nanobodies/templates/pdb4krp.ent,False


Next thing that we do - enumerate with sadie

In [10]:
from sadie.renumbering import Renumbering
renumbering_api = Renumbering(scheme="chothia", region_assign="imgt", run_multiproc=True)
template_df_numbered = renumbering_api.run_dataframe(
    templates_df,
    seq_field="sequence",
    seq_id_field="record_id",
    return_join=True
)

In [11]:
template_df_numbered = template_df_numbered[template_df_numbered.follow.apply(len) == 0].reset_index(drop=True)

In [12]:
region_columns = [
    #'fwr1_aa_gaps',
    'fwr1_aa_no_gaps',
    #'cdr1_aa_gaps',
    'cdr1_aa_no_gaps', #'fwr2_aa_gaps',
    'fwr2_aa_no_gaps', #'cdr2_aa_gaps',
    'cdr2_aa_no_gaps', #'fwr3_aa_gaps',
    'fwr3_aa_no_gaps', #'cdr3_aa_gaps',
    'cdr3_aa_no_gaps', # 'fwr4_aa_gaps',
    'fwr4_aa_no_gaps'
]
assert (template_df_numbered[region_columns].applymap(len).sum(1) == template_df_numbered.sequence_x.apply(len)).all()

In [13]:
for col in region_columns:
    colname = col.split("_")[0]
    template_df_numbered.loc[:, f'{colname}_start'] = 0
    template_df_numbered.loc[:, f'{colname}_end'] = 0

for rowid, row_data in template_df_numbered[region_columns].applymap(len).iterrows():
  starts = np.cumsum([0]+list(row_data.values))
  for col, (s, e) in zip(region_columns, zip(starts, starts[1:])):
    colname = col.split("_")[0]
    template_df_numbered.loc[rowid, f'{colname}_start'] = s
    template_df_numbered.loc[rowid, f'{colname}_end'] = e

In [14]:
template_df_numbered.to_csv(TEMPLATEDIR / "numbered_chains.csv", index=None)

In [15]:
template_df_numbered

,pdbid,record_id,chain_id,dbxrefs,sequence_x,filename,isEGFR,Id,sequence_y,domain_no,...,fwr2_start,fwr2_end,cdr2_start,cdr2_end,fwr3_start,fwr3_end,cdr3_start,cdr3_end,fwr4_start,fwr4_end
0,4krl,4KRL:B,B,"[PDB:4KRL, PDB:4KRL]",QVKLEESGGGSVQTGGSLRLTCAASGRTSRSYGMGWFRQAPGKERE...,esm3_generated_nanobodies/templates/pdb4krl.ent,False,4KRL:B,QVKLEESGGGSVQTGGSLRLTCAASGRTSRSYGMGWFRQAPGKERE...,0,...,33,50,50,58,58,96,96,113,113,122
1,4krn,4KRN:A,A,"[PDB:4KRN, PDB:4KRN]",QVQLQESGGGLVQPGGSLRLSCAASGRTFSSYAMGWFRQAPGKQRE...,esm3_generated_nanobodies/templates/pdb4krn.ent,False,4KRN:A,QVQLQESGGGLVQPGGSLRLSCAASGRTFSSYAMGWFRQAPGKQRE...,0,...,33,50,50,58,58,96,96,119,119,129
2,4krp,4KRP:B,B,"[PDB:4KRP, PDB:4KRP]",EVQLVESGGGLXXXXXXXXLSCAASGRTFSSYAMGWFRQAPGKERE...,esm3_generated_nanobodies/templates/pdb4krp.ent,False,4KRP:B,EVQLVESGGGLXXXXXXXXLSCAASGRTFSSYAMGWFRQAPGKERE...,0,...,33,50,50,58,58,96,96,116,116,124


## Template-based modelling

Now when we selected chains and defined the coordinates of the regions in each of pdb files we can try to model everything with template.

Let's do it!

In [16]:
import py3Dmol
from huggingface_hub import login
import torch

from esm.utils.structure.protein_chain import ProteinChain
from esm.models.esm3 import ESM3
from esm.sdk.api import (
    ESMProtein,
    GenerationConfig,
)

/usr/local/lib/python3.10/dist-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/usr/local/lib/python3.10/dist-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/usr/local/lib/python3.10/dist-packages/google/cloud/storage/transfer_manager.py:30: UserWarning: The module `transfer_manager` is a preview feature. Functionality and API may change. This warning will be removed in a future release.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/usr/local/lib/python3.10/dist-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subn

In [17]:
model =  ESM3.from_pretrained("esm3_sm_open_v1", device=torch.device("cuda"))

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Fetching 22 files:   0%|          | 0/22 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/3.30k [00:00<?, ?B/s]

data/ParentChildTreeFile.txt:   0%|          | 0.00/595k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

...

esm3_function_decoder_v0.pth:   0%|          | 0.00/1.30G [00:00<?, ?B/s]

esm3_sm_open_v1.pth:   0%|          | 0.00/2.80G [00:00<?, ?B/s]

esm3_structure_decoder_v0.pth:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

esm3_structure_encoder_v0.pth:   0%|          | 0.00/62.3M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/esm/pretrained.py:68: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(


In [18]:
# Here I use code from
# https://github.com/evolutionaryscale/esm/blob/a46ffdc2fc9f91cb6ac30c77d9ddfe89912f6c8a/examples/esmprotein.ipynb#L71
# currently not in use, but it might be a good idea to use it
from biotite.structure import annotate_sse

def get_approximate_ss(protein_chain: ProteinChain):
    # get biotite's ss3 representation
    ss3_arr = annotate_sse(protein_chain.atom_array)
    biotite_ss3_str = ''.join(ss3_arr)

    # translate into ESM3's representation
    translation_table = str.maketrans({
        'a': 'H', # alpha helix
        'b': 'E', # beta sheet
        'c': 'C', # coil
    })
    esm_ss3 = biotite_ss3_str.translate(translation_table)
    return esm_ss3

In [19]:
def get_new_protein_with_gaps(protein, intervals=None):
  """
  Produces template for generation with any given replacement scheme (defined by a series of intervals).

  Intervals: list of triplets (is_mask, (start, end), new_length), where:
    - is_masked defines if the region needs to be regenerated
    - (start, end) the indices of the corresponding coordinates/sequence/etc.
    - new_length defines the length of the fragment which will be generated instead of the current one
  """
  # protein.coordinates
  if intervals is None or len(intervals) == 0:
      return protein
  new_coordinates = []
  new_sequence = []
  for is_masked, (s, e), new_length in intervals:
      if is_masked:
          if new_length is None:
              new_length = e - s
          interval_coords = torch.full((new_length, 37, 3), np.nan)
          new_coordinates.append(interval_coords)
          new_sequence.append("_" * new_length)
      else:
          new_coordinates.append(protein.coordinates[s:e])
          new_sequence.append(protein.sequence[s:e])
  new_coordinates = torch.cat(new_coordinates)
  new_sequence = "".join(new_sequence)
  return ESMProtein(sequence=new_sequence, coordinates=new_coordinates)


from collections import defaultdict

def get_intervals(row_df, masked_regions={'cdr1', 'cdr2', 'cdr3'}, masked_lengths=dict()):
  """helper method to produce intervals from row with vhh information and numbering"""
  for name in vhh_region_names:
    s, e = row_df[f"{name}_start"], row_df[f"{name}_end"]
    is_masked = name in masked_regions
    yield is_masked, (s, e), masked_lengths.get(name)

# list(get_intervals(row_df, masked_lengths={'cdr3': 14}))

# get_new_protein_with_gaps(protein)
def get_length_from_normal(orig_length, width=3, num_samples=10):
  min_length = (orig_length - width if orig_length > width+3 else orig_length)
  max_length = orig_length + width
  return (np.random.normal(orig_length, width, num_samples).clip(min_length, max_length)//1).astype(int)

In [20]:
# ProteinChain.from_pdb()
num_samples = 5
# (row_df.filename, chain_id=chain_id, id=pdb_id)
run_info = []  # just in case we save some additional information to the dataframe
for rowid, row_df in template_df_numbered.iterrows():
  protein_chain = ProteinChain.from_pdb(row_df.filename, chain_id=row_df.chain_id, id=row_df.pdbid)
  protein = ESMProtein.from_protein_chain(protein_chain)
  # protein.secondary_structure = get_approximate_ss(protein_chain)
  # next we define secondary structure for the whole protein
  # - this is skipped for now
  cdr1_length = len(row_df.cdr1_aa_no_gaps)
  cdr1_length_candidates = get_length_from_normal(cdr1_length, width=1, num_samples=num_samples)

  cdr2_length = len(row_df.cdr2_aa_no_gaps)
  cdr2_length_candidates = get_length_from_normal(cdr2_length, width=2, num_samples=num_samples)

  cdr3_length = len(row_df.cdr3_aa_no_gaps)
  cdr3_length_candidates = get_length_from_normal(cdr3_length, width=3, num_samples=num_samples)
  GENPATH = SAVEDIR / f"{row_df.pdbid}_{row_df.chain_id}"
  GENPATH.mkdir(exist_ok=True)
  print(GENPATH)

  for icandidate, (cdr1, cdr2, cdr3) in enumerate(zip(cdr1_length_candidates, cdr2_length_candidates, cdr3_length_candidates)):
      new_length_values = {'cdr1': cdr1, 'cdr2': cdr2, 'cdr3': cdr3}
      protein_intervals = list(get_intervals(row_df, masked_lengths=new_length_values))
      template_protein = get_new_protein_with_gaps(protein, intervals=protein_intervals)
      num_aa_to_generate = cdr1+cdr2+cdr3

      generated_filename = GENPATH / f"nano{current_timestamp}_candidate{icandidate:04d}_{row_df.pdbid}-{row_df.chain_id}_{cdr1}-{cdr2}-{cdr3}.pdb"
      gen_protein = model.generate(template_protein, GenerationConfig(track="sequence", num_steps=num_aa_to_generate//8))
      gen_protein = model.generate(gen_protein, GenerationConfig(track="structure", num_steps=num_aa_to_generate//8))

      gen_protein.to_pdb(generated_filename.as_posix())

      run_info.append({
          'icandidate': icandidate,
          'pdbid': row_df.pdbid,
          'chain_id': row_df.chain_id,
          'template_sequence': str(template_protein.sequence),
          'num_steps': num_aa_to_generate//8,

          'old_cdr1_length': cdr1_length,
          'new_cdr1_length': cdr1,
          'old_cdr2_length': cdr2_length,
          'new_cdr2_length': cdr2,
          'old_cdr3_length': cdr3_length,
          'new_cdr3_length': cdr3,
          'filename': generated_filename.as_posix()
      })
  #break
# row_df.head()

esm3_generated_nanobodies/4krl_B


/usr/local/lib/python3.10/dist-packages/esm/pretrained.py:29: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(
/usr/local/lib/python3.10/dist-packages/

esm3_generated_nanobodies/4krn_A


100%|██████████| 4/4 [00:00<00:00,  5.95it/s]


esm3_generated_nanobodies/4krp_B


100%|██████████| 4/4 [00:00<00:00,  6.19it/s]


In [21]:
pd.DataFrame(run_info).to_csv(SAVEDIR/"gen_info.csv", index=None)

## After everything is generated, we need to download it

In [22]:
!zip -r generated.zip {SAVEDIR.as_posix()}

  adding: esm3_generated_nanobodies/ (stored 0%)
  adding: esm3_generated_nanobodies/4krp_B/ (stored 0%)
  adding: esm3_generated_nanobodies/4krp_B/nano2024-10-17_14:39:04_candidate0003_4krp-B_8-10-22.pdb (deflated 77%)
  adding: esm3_generated_nanobodies/4krp_B/nano2024-10-17_14:39:04_candidate0002_4krp-B_8-8-18.pdb (deflated 78%)
  adding: esm3_generated_nanobodies/4krp_B/nano2024-10-17_14:39:04_candidate0001_4krp-B_7-7-19.pdb (deflated 77%)
  adding: esm3_generated_nanobodies/4krp_B/nano2024-10-17_14:39:04_candidate0004_4krp-B_9-6-20.pdb (deflated 78%)
  adding: esm3_generated_nanobodies/4krp_B/nano2024-10-17_14:39:04_candidate0000_4krp-B_8-7-21.pdb (deflated 78%)
  adding: esm3_generated_nanobodies/gen_info.csv (deflated 83%)
  adding: esm3_generated_nanobodies/4krn_A/ (stored 0%)
  adding: esm3_generated_nanobodies/4krn_A/nano2024-10-17_14:39:04_candidate0004_4krn-A_8-6-21.pdb (deflated 78%)
  adding: esm3_generated_nanobodies/4krn_A/nano2024-10-17_14:39:04_candidate0002_4krn-A_7-

In [23]:
from google.colab import files
files.download('generated.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>